# HMI for the DZ Komatsu Harvester
This is a KWH HMI which allows to send S3I-B GetValueRequest to request the data from the DT Komatsu harvester. Just run the following code to prepare and send your GetValueRequest to the harvester. 

In [ ]:
import s3i
import uuid
import getpass 
import collections
import jwt 
import json
import requests
from tools import print_with_timestamp, check_message_encryption, yes, no

In [ ]:
CLIENT_ID = "s3i:da36496a-d2a8-411f-a01c-978e5e59d78a"
CLIENT_SECRET = "452d9a83-baa5-47bd-a01e-b437bfbe08f6"
ENDPOINT = "s3ib://s3i:da36496a-d2a8-411f-a01c-978e5e59d78a"

RECEIVER_ID = "s3i:37883bba-3833-4364-9bd2-dd291913b0e9"
RECEIVER_ENDPOINT = "s3ib://s3i:37883bba-3833-4364-9bd2-dd291913b0e9"


## Configure the notebook


In [ ]:
print_with_timestamp("HMI for the komatsu harvester, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=CLIENT_ID,
                                             client_secret=CLIENT_SECRET,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

## MessageID Storage

In [ ]:
messageIds = list()

## Prepare the get value request

In [ ]:
print_with_timestamp("Prepare the GetValueRequest.")
attributePath = input(
        '[S3I]: please enter the attribute path of the value you want to receive from the harvester. Examples: \n<empty path, just press enter>  \nattributes \nattributes/features\nattributes/features/ml40::Location\nattributes/features/ml40::Location/longitude\nattributes/features/ml40::Location/latitude\n').strip('," ')

msg_uuid = "s3i:" + str(uuid.uuid4())

getReq = s3i.GetValueRequest()
getReq.fillGetValueRequest(senderUUID=CLIENT_ID, receiverUUID=[RECEIVER_ID], 
                            sender_endpoint=ENDPOINT,
                            attributePath=attributePath, msgUUID=msg_uuid)
print_with_timestamp("GetValueRequest prepared:")
print(json.dumps(getReq.msg, indent=2))

## Send the request

In [ ]:
print_with_timestamp("Sending the GetValueRequest to the DT komatsu harvester")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
messageIds.append(getReq.msg["identifier"])
        
''' send S3I-B message to S3I-Broker API
'''
response = requests.post(url="https://broker.s3i.vswf.dev/"+RECEIVER_ENDPOINT,
                                data=json.dumps(getReq.msg), headers=headers)
print_with_timestamp(response.text)

## Receive the response from DT Komatsu Harvester

In [ ]:
def receive():
    print_with_timestamp("Checking the harvester's response")
    access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
    headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
    response = requests.get(url="https://broker.s3i.vswf.dev/"+ENDPOINT , headers=headers)
    msg_json = ""
    value_json = ""
    json_acceptable_string = response.text.replace("'", "\"")
    if json_acceptable_string:
        if check_message_encryption(response.text.strip('"')) == "pgp":
            print_with_timestamp("You received a PGP message but this notebook can not decrypt PGP messages. Use the 03_inbox notebook to receive PGP messages.")
            print_with_timestamp("PGP Message: " + response.text)
            repeat()
        else: 
            msg_json = json.loads(json_acceptable_string)
            if msg_json["replyingToMessage"] in messageIds:
                value_json = msg_json["value"]
                messageIds.remove(msg_json["replyingToMessage"])
    else:
        print_with_timestamp("The harvester did not respond yet.")
        repeat()
    return msg_json, value_json

def repeat():
    decision = input("[S3I] Do you want to check for new messages again? [j/n]")
    if decision in yes:
        receive()
    elif decision in no:
        print_with_timestamp("You do not want to check for more messages. If you want to check for new messages, just execute this cell again (Run button or SHIFT+RETURN)")
    else:
        print_with_timestamp("I could not understand your response. If you want to check for new messages, just execute this cell again (Run button or SHIFT+RETURN)")

In [ ]:
msg, value = receive()

In [ ]:
if msg:
    msg = json.loads(json.dumps(msg), object_pairs_hook=collections.OrderedDict)
    print_with_timestamp("You received a message: " + json.dumps(msg, indent=2))
    

In [ ]:
if value:
    value = json.loads(json.dumps(value), object_pairs_hook=collections.OrderedDict)
    print_with_timestamp("Your requested value is: " + json.dumps(value, indent=2))